In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

stores_name = []
stores_address = []
stores_phonenum = []
url = 'target url'
chromedriver = 'chromedriver경로'
options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(chromedriver, chrome_options=options)

driver.get(url)
store_menu = driver.find_elements_by_css_selector("li.menu_open")[0]
store_menu.click() #매장소개 클릭
time.sleep(0.5)
store_find = store_menu.find_elements_by_css_selector("a")[0]
store_find.click() # 매장찾기 클릭
time.sleep(0.5)

address_select = driver.find_elements_by_css_selector("ul.item3")[0]
address_select = address_select.find_elements_by_css_selector("button")[2]
address_select.click() # 지역검색 클릭
time.sleep(0.5)
print('서울시내 매장 검색')
address_select = driver.find_elements_by_css_selector("div.form_list > select.st04")[0]
address_select = driver.find_elements_by_css_selector("option")[1]
address_select.click() # 서울특별시 클릭.

time.sleep(1)
store_object = driver.find_elements_by_css_selector("ul.list02 > li")
print('매장 정보 수집 끝')
for i, j in enumerate(store_object):
    stores_name.append(j.find_element_by_css_selector("div.shop_detail02 > p").text)
    
    temp=j.find_elements_by_css_selector("div.subinfo > p")
    
    if temp[0].text == '':
        stores_address.append('주소없음')
    else:
        stores_address.append(temp[0].text)
    if temp[1].text == '':
        stores_phonenum.append('번호없음')
    else:
        stores_phonenum.append(temp[1].text)

driver.quit()

store_dict = {"매장명":stores_name, "매장주소":stores_address, "전화번호":stores_phonenum}

    

create_ex = pd.DataFrame(store_dict)

create_ex.to_csv('경로/파일명.csv',encoding='utf-8-sig')
print('매장 정보 저장 끝')

매장 정보 저장 끝


In [16]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import re

stores_name = []
stores_address = []
stores_phonenum = []
page_start=3
url = 'target url'
chromedriver = 'chromedriver경로'
t = True
options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(chromedriver, chrome_options=options)
# driver = webdriver.Chrome(chromedriver)

driver.get(url)
store_menu = driver.find_elements_by_css_selector("select#locationSelect > option")[1]

store_menu.click() #매장소개 클릭
time.sleep(0.5)
driver.find_element_by_css_selector("div.locationSelectWrap > a").click()
time.sleep(1.5)

end_page = True
while end_page:
    if stores_name != []:
        page_start = 2
    for i in driver.find_elements_by_css_selector("tr.shopSearch"):
        idx, stroe_code = tuple(re.sub("(goView\()*(\); return false;)*(\,)*(\')*","",i.find_element_by_css_selector("a").get_attribute('onclick')).split())
        response = requests.get("http://www.lotteria.com/Shop/Shop_View.asp?Idx="+idx+"&StoreCode="+stroe_code)
        soup = BeautifulSoup(response.text, "html.parser")
        
        stores_name.append(soup.select_one("div#content div.h3_box").text.strip())
        stores_address.append(soup.select_one("td.rt").text.strip())
        stores_phonenum.append(re.sub("( )*","",soup.select_one("td.phone").text.split("/")[0]))

    driver.find_elements_by_css_selector("div.paging_basic a")[2].click()
    
    time.sleep(2.5)
    for next_page in range(page_start,12):
        page_numbers = driver.find_elements_by_css_selector("div.paging_basic a")
        for i in driver.find_elements_by_css_selector("tr.shopSearch"):
            idx, stroe_code = tuple(re.sub("(goView\()*(\); return false;)*(\,)*(\')*","",i.find_element_by_css_selector("a").get_attribute('onclick')).split())
            response = requests.get("http://www.lotteria.com/Shop/Shop_View.asp?Idx="+idx+"&StoreCode="+stroe_code)
            soup = BeautifulSoup(response.text, "html.parser")
            if re.sub("( )*","",soup.select_one("td.phone").text.split("/")[0]) in stores_phonenum:
                end_page = False
                print('수집 끝')
                break
            stores_name.append(soup.select_one("div#content div.h3_box").text.strip())
            stores_address.append(soup.select_one("td.rt").text.strip())
            stores_phonenum.append(re.sub("( )*","",soup.select_one("td.phone").text.split("/")[0]))

        if end_page == False:
            break
        page_numbers[next_page].click()
        time.sleep(2.5)
    
store_dict = {"매장명":stores_name, "매장주소":stores_address, "전화번호":stores_phonenum}
create_ex = pd.DataFrame(store_dict)

create_ex.to_csv('경로/파일명.csv',encoding='utf-8-sig')
print('매장 정보 저장 끝')
driver.quit()



수집 끝
매장 정보 저장 끝
